# Agenda

- 工廠監控

    - 今天的主軸：如何在Raspberry Pi上使用python和PLC、電表溝通, 並把資料送上雲端

- Modbus
    - 何謂Modbus?
    - modbus格式介紹
    - modbus常被採用的原因
        
- Modbus套件
    - 可用的套件
    - 安裝
    - 操作

- PLC
    - 何謂PLC
    - 如何通訊、控制
    - 來看看PLC的Modbus點表
    - Live Demo

- Power Meter
    - 何謂power meter
    - 來看看Power Meter的點表
    - Live Demo
    
- MQTT - 即時性應用
    - Mosquitto
    - paho
    
- The End!?
 
- Q&A

----


## 工廠監控

----


### 今天的主軸：如何使用python和PLC、電表、逆變器溝通, 並把資料送上雲端

- 範例可以在windows、Linux、Raspberry pi、BeagleBone Black上執行
    
- ![PLC_Meter](image/plc_meter.png)

## Live Demo

先來看看我們使用Python可以達到的效果

- Python + PLC + Power Meter + MQTT + Linear MQTT Dashboard

![mqtt](image/mqtt.png)

----

## 工廠監控的現況

先看看業界現在怎麼做…

----


### 很常見的方式

- SCADA + HMI + PLC + IO module

- PLC + SCADA


### 系統小一點的話

- HMI + PLC + IO module




### 喜歡自已來!?

- 當然是自己開發囉! VB.Net, C#, C 各種語言不拘!


### 為何SCADA這麼常出現?

- 因為工業控制需要快速又穩定的RAD工具，讓每個人都能做出一定水準之上的系統

![Scada_std_anim_no_lang](image/Scada_std_anim_no_lang.gif)


----

## Modbus

一個工業控制一定不能錯過的協定

----


### 何謂 Modbus?

一種工業控制中很常用的通訊協定

- [維基百科怎麼說](https://zh.wikipedia.org/wiki/Modbus)?

- 為何介紹Modbus

![協定文字雲](image/protocol_cloud.png)
       

### modbus格式介紹 

[參考](http://gridconnect.com/blog/tag/modbus-rtu/)

- Modbus/RTU: [start time] [Address 8bits + Function 8bits + Data Nx8bits + CRC 16bits] [End time]

- Modbus/TCP: [header 6byte + Address 8bits + Function 8bits + Data Nx8bits]

- Modbus/ASCII: 現在比較少人用，跳過不講

![格式](image/MODBUS-Frame.png)
        


### modbus常被採用的原因
        
- 公開發表並且無版稅要求

- 相對容易的工業網路部署

- 協定格式簡單，極省資源


----

## Modbus套件

----



### 網路上較多人提到的三個套件

[performance比較](https://stackoverflow.com/questions/17081442/python-modbus-library)

- modbus-tk: Modbus/RTU, Modbus/TCP

- pymodbus: 據說實作最完整，但使用資源相對的多，相依套件也多

- MinimalModbus: Modbus/RTU, Modbus/ASCII



### Modbus-tk

個人推薦使用

- 安裝方式
    - pip install serial
    - pip install modbus_tk

- 操作
    - [Python與PLC共舞](https://github.com/maloyang/PLC-Python)
     

----

## PLC

工業控制常用的元素，[看看wiki怎麼說](https://zh.wikipedia.org/wiki/%E5%8F%AF%E7%BC%96%E7%A8%8B%E9%80%BB%E8%BE%91%E6%8E%A7%E5%88%B6%E5%99%A8)

----



### 通訊方式

- 自有協定

- <h3>Modbus</h3>

- CAN

- ...etc


### PLC的Modbus點表

![PLC點表](image/fatek_modbus_addr.png)

- [Live Demo](Modbus.ipynb)

In [1]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time


In [2]:
mbComPort = "COM8"   # your RS-485 port. for linux --> "/dev/ttyUSB0"
baudrate = 9600
databit = 8 #7, 8
parity = 'N' #N, O, E
stopbit = 1 #1, 2
mbTimeout = 100 # ms


In [3]:

mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 1
addr = 2 #base0 --> my 110V Led燈泡

for i in range(5):
    try:
        value = i%2
        #-- FC5: write multi-coils
        rr = master.execute(mbId, cst.WRITE_SINGLE_COIL, addr, output_value=value)
        print("Write(addr, value)=",  rr)

    except Exception as e:
        print("modbus test Error: " + str(e))

    time.sleep(2)

master._do_close()


Write(addr, value)= (2, 0)
Write(addr, value)= (2, 65280)
Write(addr, value)= (2, 0)
Write(addr, value)= (2, 65280)
Write(addr, value)= (2, 0)


True

----

## Power Meter

電力監控常見的元素

----



### Power Meter的點表

![Power Meter的點表](image/power_meter.gif)



### Power Meter的浮點數表示方式

![浮點數](image/power_meter_float.gif)

----
![float](image/mb_float.png)
- [Live Demo](Modbus.ipynb)


In [4]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time
from struct import *

In [5]:
mbComPort = 'COM8' #your RS-485 port #'/dev/ttyUSB0' for linux(RPi3)
baudrate = 9600
databit = 8
parity = 'N'
stopbit = 1
mbTimeout = 100 # ms

In [6]:
mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 4
addr = 0x1000 # power-meter is base 0
# notice: meter not support FC6, only FC16

try:
    # FC3
    rr = master.execute(mbId, cst.READ_INPUT_REGISTERS, addr, 2)
    print('read value:', rr)

    # convert to float:
    # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
    try:
        v_a_hi = rr[1]
        v_a_lo = rr[0]
        v_a = unpack('>f', pack('>HH', v_a_hi, v_a_lo))
        print('v_a=', v_a)
        #struct.unpack(">f",'\x42\xd8\x6b\x8d')
    except Exception as e:
        print(e)

except Exception as e:
    print("modbus test Error: " + str(e))


master._do_close()


read value: (39507, 17124)
v_a= (114.3014144897461,)


True

### 讀回電錶的功率

In [7]:
mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 4
addr = 0x1034 #kWh

try:
    # FC3
    rr = master.execute(mbId, cst.READ_INPUT_REGISTERS, addr, 2)
    print('read value:', rr)

    # convert to float:
    # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
    try:
        hi = rr[1]
        lo = rr[0]
        kwh = unpack('>f', pack('>HH', hi, lo))
        print('kWh=', kwh)
    except Exception as e:
        print(e)

except Exception as e:
    print("modbus test Error: " + str(e))


master._do_close()


read value: (3077, 18333)
kWh= (80408.0390625,)


True

### 多個參數一起讀: Vin_a, VIn_avg, freq.

In [8]:
mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 4
#[0x1000-0x1001]=VIn_a
addr = 0x1000#4096

try:
    # FC3
    rr = master.execute(mbId, cst.READ_INPUT_REGISTERS, addr, 26)
    print('read value:', rr)

    # convert to float:
    # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
    try:
        # VIn_a
        v_a_hi = rr[1]
        v_a_lo = rr[0]
        v_a = unpack('>f', pack('>HH', v_a_hi, v_a_lo))
        print('v_a=', v_a)
        #struct.unpack(">f",'\x42\xd8\x6b\x8d')
        
        # VIn_avg
        v_hi = rr[7]
        v_lo = rr[6]
        v_avg = unpack('>f', pack('>HH', v_hi, v_lo))
        print('v_avg=', v_avg)
        
        # Frequency
        freq_hi = rr[25]
        freq_lo = rr[24]
        freq = unpack('>f', pack('>HH', freq_hi, freq_lo))
        print('Frequency=', freq)
        
    except Exception as e:
        print(e)

except Exception as e:
    print("modbus test Error: " + str(e))


master._do_close()


read value: (10639, 17125, 0, 0, 0, 0, 10639, 17125, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24794, 17008)
v_a= (114.58116912841797,)
v_avg= (114.58116912841797,)
Frequency= (60.094581604003906,)


True

----

## MQTT - 即時性應用

已經可以採集資料了，來談談怎麼上傳雲端吧

----


### Mosquitto

一個broker套件，當然也可以做為client使用

- 以NB X260來說，4000多個連結沒有問題

![Mosquitto](image/Eclipse-Mosquitto-logo.png)


### paho

便利的MQTT client端套件

- [link](https://pypi.org/project/paho-mqtt/)
- install: `pip install paho-mqtt`

![paho](image/mqtt-paho-featured-image.jpg)


### live demo

- 以先架好的rpi3控制PLC為例


In [9]:
import paho.mqtt.client as mqtt  #import the client1
import time

import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time

mbComPort = '/dev/ttyUSB0'
baudrate = 9600
databit = 8
parity = 'N'
stopbit = 1
mbTimeout = 100 # ms

def on_connect(client, userdata, flags, rc):
    m="Connected flags"+str(flags)+", result code "+str(rc)+", client_id  "+str(client)
    print(m)

    # first value OFF
    print('set light off!')
    control_light(0)
    client1.publish(topic,0)    

def on_message(client1, userdata, message):
    print("message received  "  ,str(message.payload.decode("utf-8")), message.topic, message.qos, message.retain)
    if message.topic == topic:
        my_message = str(message.payload.decode("utf-8"))
        print("set light: ", my_message)
        if my_message=='1' or my_message==1:
            control_light(1)
        else:
            control_light(0)

def on_log(client, userdata, level, buf):
    print("log: ",buf)

def control_light(value):
    mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
    master = modbus_rtu.RtuMaster(mb_port)
    master.set_timeout(mbTimeout/1000.0)

    mbId = 1
    addr = 2 #base0

    try:
        #-- FC5: write multi-coils
        rr = master.execute(mbId, cst.WRITE_SINGLE_COIL, addr, output_value=value)
        print("Write(addr, value)=%s" %(str(rr)))

    except Exception as e:#except Exception, e:
        print("modbus test Error: " + str(e))

    master._do_close()


# some online free broker:
#   iot.eclipse.org
#   test.mosquitto.org
#   broker.hivemq.com
broker_address="iot.eclipse.org"
topic = "malo-iot/light"
client1 = mqtt.Client()    #create new instance
client1.on_connect = on_connect        #attach function to callback
client1.on_message = on_message        #attach function to callback
#client1.on_log=on_log

time.sleep(1)
client1.connect("iot.eclipse.org", 1883, 60)      #connect to broker
#client1.loop_start()    #start the loop
client1.subscribe(topic)

client1.loop_forever()

Connected flags{'session present': 0}, result code 0, client_id  <paho.mqtt.client.Client object at 0x70ccb4b0>
set light off!
Write(addr, value)=(2, 0)
message received   0 malo-iot/light 0 1
set light:  0
Write(addr, value)=(2, 0)
message received   0 malo-iot/light 0 0
set light:  0
Write(addr, value)=(2, 0)
message received   1 malo-iot/light 0 0
set light:  1
Write(addr, value)=(2, 65280)
message received   0 malo-iot/light 0 0
set light:  0
Write(addr, value)=(2, 0)
message received   1 malo-iot/light 0 0
set light:  1
Write(addr, value)=(2, 65280)
message received   0 malo-iot/light 0 0
set light:  0
Write(addr, value)=(2, 0)


KeyboardInterrupt: 

----

## The End!!

----


----

## Q&A

----